In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import torch
import numpy as np
import re
import string
import asyncio
import os
import copy
from time import time
from transformers import AutoTokenizer

In [ ]:
from hfppl import Model, CachedCausalLM, LMContext, smc_standard
from hfppl.distributions import TokenCategorical

In [ ]:
# Load the language model. 
# Mistral and Vicuna are open models; to use a model with restricted access, like LLaMA 2,
# pass your HuggingFace API key as the optional `auth_token` argument:
# LLM = CachedCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf", auth_token=os.environ['HF_AUTH_TOKEN'])
# LLM = CachedCausalLM.from_pretrained("lmsys/vicuna-7b-v1.5")
# LLM = CachedCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1")
MODEL_ID = "codellama/CodeLlama-7b-Instruct-hf"

In [ ]:
LLM = CachedCausalLM.from_pretrained(MODEL_ID, load_in_8bit=True)
#LLM.batch_size = 10

/home/timv/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True, prefix_token=None, middle_token=None, suffix_token=None, eot_token=None, fill_token=None)

In [ ]:
prompt = """
You have access to a political survey data table named "data", which includes the following columns:
- "age" (integer)
- "gender" ("male" or "female"),
- "year" (integer)
- "state_color" ("blue" or "red")
- "zipcode" (integer)
- "vote" ("democrat" or "republican") 
- "race_ethnicity" ("white", "black", or "latino").

Q: Write a SQL query that shows individuals' age and gender, for people over 50 years old.
A: SELECT age, gender FROM data WHERE age>50 </s>
Q: Write a SQL query that shows individuals' vote and zipcode, ordered from lowest to highest age.
A: SELECT vote, zipcode, age FROM data ORDER BY age ASC </s>
Q: Write a SQL query that returns white voters' average age for each state color.
A:""" 

In [ ]:
#LLM.cache_kv(LLM.tokenizer.encode(prompt))

In [ ]:
context = prompt+" SELECT state_color"

In [ ]:
from genparse import Float

In [ ]:
class GreedilyTokenizedLLM:

    def __init__(self, llm, tokenizer):
        self.tokenizer = tokenizer
        self._model = llm

        self._decode = [self.tokenizer.decode([i]) for i in range(self.tokenizer.vocab_size)]
        self.V = set(self._decode)
        self.eos = self.tokenizer.eos_token

    def __call__(self, xs):
        return self.model(self.tokenizer.encode(xs))

    def p_next(self, xs, top=None):
        return asyncio.run(self._p_next(xs, top=top))

    async def _p_next(self, xs, top=None):
        assert isinstance(xs, str)
        tokens = self.tokenizer.encode(xs)

        _logp = await self._model.next_token_logprobs(tokens)
        _p = np.exp(_logp)

        if top is None:
            top_p = _p.argsort()
        else:
            top_p = _p.argsort()[-top:]
        pp = Float.chart()
        for i in reversed(top_p):
            pp[self._decode[i]] = _p[i]
        if top is None:
            return pp
        else:
            return pp.normalize()


In [75]:
from genparse.experimental.earley import Earley
from genparse.lm import LM
from genparse import Boolean, EOS, add_EOS

In [77]:
class EarleyBoolMaskCFGLM(LM):
    "LM-like interface for Boolean-masking CFG models."

    def __init__(self, cfg):
        if EOS not in cfg.V: cfg = add_EOS(cfg)
        if cfg.R != Boolean: cfg = cfg.map_values(lambda x: Boolean(x>0), Boolean)
        self.model = Earley(cfg.prefix_grammar.renumber().nullaryremove().unarycycleremove().renumber())
        super().__init__(eos = self.model.eos, V = self.model.V)

    def p_next(self, context):
        p = self.model.p_next(context).trim()
        return Float.chart({w: 1 for w in p})

    def __call__(self, context):
        return float(self.model(context) != Boolean.zero)


In [76]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
m = GreedilyTokenizedLLM(LLM, tokenizer)

In [ ]:
m.p_next(context, top=10)

key,value
",",0.069331209816617
AS,0.668154972391995
as,0.19444419189443085
.,0.021477806567383823
,0.014532619249118384
_,0.010224974054857044
"""",0.007658152378803844
",""",0.007364772352179361
state,0.006811301294614755


In [ ]:
from genparse.proposal import CharacterProposal, TokenProposal
from genparse.cfglm import BoolMaskCFGLM, locally_normalize
from genparse.util import LarkStuff

In [78]:
guide = EarleyBoolMaskCFGLM(LarkStuff(r"""
start: WS? "SELECT" WS select_expr WS "FROM" WS from_expr [WS "WHERE" WS bool_condition] [WS "GROUP BY" WS var_list] [WS "ORDER BY" WS orderby_expr] WS EOS
EOS: "</s>"
select_expr: STAR | select_list
bool_condition: bool_expr | "(" bool_condition WS "AND" WS bool_condition ")" | "(" bool_condition WS "OR" WS bool_condition ")"
bool_expr: var "=" value | var ">" value | var "<" value
from_expr: "data"
orderby_expr: var_list WS "ASC" | var_list WS "DESC"
select_list: select_var ("," WS select_var)*
var_list: var ("," WS var)*
select_var: var | "AVG(" var ")" | "MEDIAN(" var ")" | "COUNT(" var ")"
var: "age" | "gender" | "year" | "state_color" | "zipcode" | "vote" | "race_ethnicity"
value: NUMBER | "'red'" | "'blue'" | "'white'" | "'black'" | "'latino'" | "'republican'" | "'democrat'" | "'male'" | "'female'"
STAR: "*"
NUMBER: /\d+/
WS: /[ \n]/

""").char_cfg(.99, ignore='[ ]?'))

In [79]:
proposal = CharacterProposal(llm=m, guide=guide)

In [80]:
proposal.sample(prompt, verbosity=1, max_tokens=50)


| |SELECT|
| |age|
|FROM|
|data|
|WHERE|
|vote| |='|white|||'|
|GROUP| |BY|
|state||_|color|
| |ORDER| |BY|
|state|_|color|
|ASC|
| |</|s||>| |
guided-sample is 3.2075x faster than llm (p=0.07926, median: llm: 0.285727, guided-sample: 0.0890797)


("\n SELECT\n age\nFROM\ndata\nWHERE\nvote ='white'\nGROUP BY\nstate_color\n ORDER BY\nstate_color\nASC\n </s> ",
 6.841781492459149e-10)

Q: Write a SQL query that returns white voters' average age for each state color.

A: SELECT state_color, AVG(age) FROM data WHERE race_ethnicity = 'white' GROUP BY state_color `</s>`


In [81]:
token_proposal = TokenProposal(llm=m, guide=guide)

In [82]:
token_proposal.sample(prompt, verbosity=1, max_tokens=50)

 |SELECT|
| |vot||e|
|FROM|
|data|
|WHERE|
|state||||_|color|='|blue||||'|
|GROUP| |BY|
|state|||_|colo||r|
| |ORDER| |BY|
|age|
|DESC|
| |</|


(" SELECT\n vote\nFROM\ndata\nWHERE\nstate_color='blue'\nGROUP BY\nstate_color\n ORDER BY\nage\nDESC\n </",
 0.0)

In [ ]:
guide("""SELECT state_color, AVG(age) FROM data WHERE race_ethnicity = 'white' GROUP BY state_color </s>""" + guide.eos)

1.0

In [ ]:
class PureModel(Model):
    def __init__(self, prompt, max_tokens):
        super().__init__()
        self.context = LMContext(LLM, prompt)
        self.max_tokens = max_tokens
        
    async def step(self):
        logprobs = self.context.next_token_logprobs.copy()
        proposal = TokenCategorical(LLM, logprobs)     
        
        token = await self.sample(self.context.next_token(), proposal=proposal)
        self.score(logprobs[token.token_id])
        self.max_tokens -= 1
        
        # Check if done
        if token == LLM.tokenizer.eos_token_id or self.max_tokens == 0:
            self.finish()
            return

In [ ]:
async def main():
    constraint_model = PureModel(prompt, 50)
    particles = await smc_standard(constraint_model, 10)
    return particles

In [ ]:
before = time()
particles = await main()
took = time() - before

In [ ]:
print(f'{sum(len(p.context.tokens) for p in particles)/took:.2f} tokens/sec')

34.49 tokens/sec


In [ ]:
for p in particles:
    print(np.exp(p.weight), repr(str(p.context)))

4.859547066737412e-13 " SELECT state_color, AVG(age) FROM data WHERE race_ethnicity = 'white' GROUP BY state_color\n\nQ: Write a SQL query that returns the state color and average zipcode for democrat votes.\n"
3.97415483928935e-09 " SELECT state_color, AVG(age) FROM data WHERE race_ethnicity = 'white' GROUP BY state_color\n\nQ: Write a SQL query that returns the state color with most white voters.\nA: SELECT"
3.97415483928935e-09 " SELECT state_color, AVG(age) FROM data WHERE race_ethnicity = 'white' GROUP BY state_color\n\nQ: Write a SQL query that returns the state color with most white voters.\nA: SELECT"
4.1457076307306947e-10 " SELECT state_color, AVG(age) FROM data WHERE race_ethnicity = 'white' GROUP BY state_color\n\nQ: Write a SQL query that returns the state color and the number of red voters in each state"
3.97415483928935e-09 " SELECT state_color, AVG(age) FROM data WHERE race_ethnicity = 'white' GROUP BY state_color\n\nQ: Write a SQL query that returns the state color wit

In [ ]:
string_tokens = [LLM.tokenizer.decode([i]) for i in range(len(LLM.tokenizer.vocab))]